In [1]:
import ee
import ee.batch
import pyperclip

In [2]:
# Initialize the Earth Engine library.
ee.Authenticate()
ee.Initialize(project='pixelcampione-satelital2map')


In [3]:
import ee.data


satellital_ids = []
elevation_ids = []

In [4]:
import ee.batch


def export_elevation(dataset_image,area_of_interest, filename, copy: bool = True):
    dataset = ee.image.Image(dataset_image)

    selected = dataset.select('elevation')
    # Define export parameters.
    task = ee.batch.Export.image.toDrive(**{
        'image': selected,
        'description': 'elevation_to_drive',
        'fileNamePrefix': filename,
        'scale': 90,  # Adjust scale if necessary
        'region': area_of_interest,
        'fileFormat': 'GeoTIFF',
        'folder': 'dataset'  # Specify the Google Drive folder name here.
    })

    # Start the export task.
    task.start()

    # Print the task ID
    print(f'Task ID: {task.id}')
    elevation_ids.append(task.id)


def export_rgb(image: ee.image.Image,area_of_interest, filename, copy: bool = True):
        
    # Select the RGB bands
    selected = image.select(['B4', 'B3', 'B2'])

    # Define export parameters.
    task = ee.batch.Export.image.toDrive(**{
        'image': selected,
        'description':'rgb_to_drive',
        'fileNamePrefix': filename,
        'scale': 30,  # Adjust scale if necessary
        'region': area_of_interest,
        'fileFormat': 'GeoTIFF',
        'folder': 'dataset'  # Specify the Google Drive folder name here.
    })

    # Start the export task.
    task.start()

    # Print the task ID
    print(f'Task ID: {task.id}')
    satellital_ids.append(task.id)

In [5]:
center = {
    'longitude': -120.0,
    'latitude': 40.0
}
offset = 0.10

# 5 x 5
lat_mult = 1
lon_mult = 3
index = {
    'start_lat': 6*lat_mult,
    'start_lon': 6*lon_mult,
    'end_lat': 6*(lat_mult+1),
    'end_lon': 6*(lon_mult+1)
}
# Me quede en 145
temp = 289

In [6]:
counter = temp
for i in range(index['start_lat'],index['end_lat']):
    for j in range(index['start_lon'], index['end_lon']):
        # Define an area of interest (for example, a bounding box around Mt. Everest).
        start = ee.geometry.Geometry.Rectangle([center['longitude'] - offset + 2*offset*j, center['latitude'] - offset - 2*offset*i
                                                , center['longitude'] + offset+ 2*offset*j, center['latitude'] + offset - 2*offset*i])
        # Call the function with your area of interest and desired filename prefix.
        export_elevation("CGIAR/SRTM90_V4",start, f'elevation_{counter}')
        counter += 1

        print("j = ",j)
    print("i = ",i)

print("-------------------------ELEVATION FINISHED-------------------------")

Task ID: NGACPOK5MTYCFDDTGAPC6KEW
j =  18
Task ID: IYLZGIXNP6D57QVTDSL4ZKQC
j =  19
Task ID: T3ITWYOAZJEG2G4OII2ZHBIQ
j =  20
Task ID: WJQ77A3CEZYXZEGID2H62X3U
j =  21
Task ID: COIHGGB2IHIYVWI74TGFRC4K
j =  22
Task ID: TNQIURCABHLUP5QY7SK572XL
j =  23
i =  6
Task ID: KMH6W5VO3VWQDFP5CQJFRXYF
j =  18
Task ID: 7R4Q54MCAZ5KPDU7JON6H7KB
j =  19
Task ID: LKITN7MXKXA2JVOKPXMO74R2
j =  20
Task ID: BILVWDKOOJYKI3WODK5IRDKS
j =  21
Task ID: YXX46UEYGVI2SAAMA6Q45RKM
j =  22
Task ID: PFUCXUPKVFEPRBMC7FEDEAAF
j =  23
i =  7
Task ID: JYEU5LD534ER4MOFB6LWGR3W
j =  18
Task ID: K2QHRU2LGI7T2FUYAO4JU6FC
j =  19
Task ID: PIVDMNFODVBYICZAMVEEU3XC
j =  20
Task ID: FMGFCGZYIFNBECMFQYQXSZJD
j =  21
Task ID: FUKDEQB3K7QTIF7MCF6YUPBG
j =  22
Task ID: EPBALYTIHZXATGUV7ERFCOKS
j =  23
i =  8
Task ID: QZKM32YZ6FTO6STYSEPSV3AR
j =  18
Task ID: O3OHCSYGEGPEA2TDNVJI6Q7L
j =  19
Task ID: 2TJ4BTWOCNH63KA3H4JLNLJL
j =  20
Task ID: DOAMQA4VHZHUOCHCSU3ZG4SU
j =  21
Task ID: MKXMURLVFWOCOHRMFU7O7GD6
j =  22
Task ID: DSEE

In [7]:
counter = temp
for i in range(index['start_lat'],index['end_lat']):
    for j in range(index['start_lon'], index['end_lon']):
        # Define an area of interest (for example, a bounding box around Mt. Everest).
        start = ee.geometry.Geometry.Rectangle([center['longitude'] - offset + 2*offset*j, center['latitude'] - offset + 2*offset*i
                                                , center['longitude'] + offset+ 2*offset*j, center['latitude'] + offset + 2*offset*i])

        # Convert ImageCollection to Image
        image = ee.imagecollection.ImageCollection("LANDSAT/LC09/C02/T1").filterBounds(start).mosaic()
        
        # Call the function with your area of interest and desired filename prefix.
        export_rgb(image, start, f'satellite_{counter}')
        counter += 1
        

        print("j = ",j)
    print("i = ",i)

print("-------------------------SATELLITE FINISHED-------------------------")

Task ID: RC55ES2D7WDUDVR7XQ5ZHE2N
j =  18
Task ID: PXR7S24Y6G7AGABIBS2DVWP7
j =  19
Task ID: LWJBC2HYD7425FLHVK6UPGOT
j =  20
Task ID: PAFDQOE2DME5QFLCM4PG3SSN
j =  21
Task ID: ACP3UGYG3BJNLSP3WJQCEHRP
j =  22
Task ID: R4WFJEXUKJWBIRKPE73GBAKP
j =  23
i =  6
Task ID: CKGTPXJQ7LAGTK3KUSTQZNI5
j =  18
Task ID: EXURTKPXIDY4UVGNGWJIHIIX
j =  19
Task ID: PF7QC5COCWODGGNFD32X5L3O
j =  20
Task ID: QOK6VW4EMDXVYNUSWFJJM65I
j =  21
Task ID: ENQJJW2S5SZYCYEXTSGQHKL3
j =  22
Task ID: KIFKZPBVCUVHNX7LAKDYPOE7
j =  23
i =  7
Task ID: YCGVY64C7SIUEICUEBZUKYHK
j =  18
Task ID: XBP7GJBJ6RINUZYM5GY754TU
j =  19
Task ID: 7SCTZ3HDKBKTYNLKMV2N7PNV
j =  20
Task ID: KL3CNBPZXTRMZHLOZ7C5LRDF
j =  21
Task ID: YIKS2UNDHRCJGYT2LKEMG4JB
j =  22
Task ID: 62VYHREB66IEFZSWXI3NLXSK
j =  23
i =  8
Task ID: 2I7PMG5JJLQ6CBNGWUPOCKCM
j =  18
Task ID: OHKDIR7RMDINQ4URVPE3BET5
j =  19
Task ID: ASIBSPK5A2BBS6GUOMHA42TC
j =  20
Task ID: 5AE42RYU6QWAZGRP5PKCSVQO
j =  21
Task ID: ZVNZDCC3WSZKMZIJIADQDOZK
j =  22
Task ID: WUTQ

In [8]:
print("Export finished!")
pyperclip.copy("\'satellital\': " + str(satellital_ids)+"," +"\n"
               "\'elevation\': " + str(elevation_ids) + "\n")


Export finished!
